### Import library

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import time
from env import Environment

### Environment

In [ ]:
class Robot: 
    def __init__(self, position): 
        self.position = position
        self.carrying = 0

class Package: 
    def __init__(self, start, start_time, target, deadline, package_id): 
        self.start = start
        self.start_time = start_time
        self.target = target
        self.deadline = deadline
        self.package_id = package_id
        self.status = 'None' # Possible statuses: 'waiting', 'in_transit', 'delivered'

class Environment: 

    def __init__(self, map_file, max_time_steps = 100, n_robots = 5, n_packages=20,
             move_cost=-0.01, delivery_reward=10., delay_reward=1., 
             seed=2025): 
        """ Initializes the simulation environment. :param map_file: Path to the map text file. :param move_cost: Cost incurred when a robot moves (LRUD). :param delivery_reward: Reward for delivering a package on time. """ 
        self.map_file = map_file
        self.grid = self.load_map()
        self.n_rows = len(self.grid)
        self.n_cols = len(self.grid[0]) if self.grid else 0 
        self.move_cost = move_cost 
        self.delivery_reward = delivery_reward 
        self.delay_reward = delay_reward
        self.t = 0 
        self.robots = [] # List of Robot objects.
        self.packages = [] # List of Package objects.
        self.total_reward = 0

        self.n_robots = n_robots
        self.max_time_steps = max_time_steps
        self.n_packages = n_packages

        self.rng = np.random.RandomState(seed)
        self.reset()
        self.done = False
        self.state = None

    def load_map(self):
        """
        Reads the map file and returns a 2D grid.
        Assumes that each line in the file contains numbers separated by space.
        0 indicates free cell and 1 indicates an obstacle.
        """
        grid = []
        with open(self.map_file, 'r') as f:
            for line in f:
                # Strip line breaks and split into numbers
                row = [int(x) for x in line.strip().split(' ')]
                grid.append(row)
        return grid
    
    def is_free_cell(self, position):
        """
        Checks if the cell at the given position is free (0) or occupied (1).
        :param position: Tuple (row, column) to check.
        :return: True if the cell is free, False otherwise.
        """
        r, c = position
        if r < 0 or r >= self.n_rows or c < 0 or c >= self.n_cols:
            return False
        return self.grid[r][c] == 0

    def add_robot(self, position):
        """
        Adds a robot at the given position if the cell is free.
        :param position: Tuple (row, column) for the robot's starting location.
        """
        if self.is_free_cell(position):
            robot = Robot(position)
            self.robots.append(robot)
        else:
            raise ValueError("Invalid robot position: must be on a free cell not occupied by an obstacle or another robot.")

    def reset(self):
        """
        Resets the environment to its initial state.
        Clears all robots and packages, and reinitializes the grid.
        """
        self.t = 0
        self.robots = []
        self.packages = []
        self.total_reward = 0
        self.done = False
        self.state = None

        # Reinitialize the grid
        #self.grid = self.load_map(sel)
        # Add robots and packages
        tmp_grid = np.array(self.grid)
        for i in range(self.n_robots):
            # Randomly select a free cell for the robot
            position, tmp_grid = self.get_random_free_cell(tmp_grid)
            self.add_robot(position)
        
        N = self.n_rows
        list_packages = []
        for i in range(self.n_packages):
            # Randomly select a free cell for the package
            start = self.get_random_free_cell_p()
            while True:
                target = self.get_random_free_cell_p()
                if start != target:
                    break
            
            to_deadline = 10 + self.rng.randint(N/2, 3*N)
            if i <= min(self.n_robots, 20):
                start_time = 0
            else:
                start_time = self.rng.randint(1, self.max_time_steps)
            list_packages.append((start_time, start, target, start_time + to_deadline ))

        list_packages.sort(key=lambda x: x[0])
        for i in range(self.n_packages):
            start_time, start, target, deadline = list_packages[i]
            package_id = i+1
            self.packages.append(Package(start, start_time, target, deadline, package_id))

        return self.get_state()
    
    def get_state(self):
        """
        Returns the current state of the environment.
        The state includes the positions of robots and packages.
        :return: State representation.
        """
        selected_packages = []
        for i in range(len(self.packages)):
            if self.packages[i].start_time == self.t:
                selected_packages.append(self.packages[i])
                self.packages[i].status = 'waiting'

        state = {
            'time_step': self.t,
            'map': self.grid,
            'robots': [(robot.position[0] + 1, robot.position[1] + 1,
                        robot.carrying) for robot in self.robots],
            'packages': [(package.package_id, package.start[0] + 1, package.start[1] + 1, 
                          package.target[0] + 1, package.target[1] + 1, package.start_time, package.deadline) for package in selected_packages]
        }
        return state
        

    def get_random_free_cell_p(self):
        """
        Returns a random free cell in the grid.
        :return: Tuple (row, col) of a free cell.
        """
        free_cells = [(i, j) for i in range(self.n_rows) for j in range(self.n_cols) \
                      if self.grid[i][j] == 0]
        i = self.rng.randint(0, len(free_cells))
        return free_cells[i]


    def get_random_free_cell(self, new_grid):
        """
        Returns a random free cell in the grid.
        :return: Tuple (row, col) of a free cell.
        """
        free_cells = [(i, j) for i in range(self.n_rows) for j in range(self.n_cols) \
                      if new_grid[i][j] == 0]
        i = self.rng.randint(0, len(free_cells))
        new_grid[free_cells[i][0]][free_cells[i][1]] = 1
        return free_cells[i], new_grid

    
    def step(self, actions):
        """
        Advances the simulation by one timestep.
        :param actions: A list where each element is a tuple (move_action, package_action) for a robot.
            move_action: one of 'S', 'L', 'R', 'U', 'D'.
            package_action: '1' (pickup), '2' (drop), or '0' (do nothing).
        :return: The updated state and total accumulated reward.
        """
        r = 0
        if len(actions) != len(self.robots):
            raise ValueError("The number of actions must match the number of robots.")

        #print("Package env: ")
        #print([p.status for p in self.packages])

        # -------- Process Movement --------
        proposed_positions = []
        # For each robot, compute the new position based on the movement action.
        old_pos = {}
        next_pos = {}
        for i, robot in enumerate(self.robots):
            move, pkg_act = actions[i]
            new_pos = self.compute_new_position(robot.position, move)
            # Check if the new position is valid (inside bounds and not an obstacle).
            if not self.valid_position(new_pos):
                new_pos = robot.position  # Invalid moves result in no change.
            proposed_positions.append(new_pos)
            old_pos[robot.position] = i
            next_pos[new_pos] = i

        moved_robots = [0 for _ in range(len(self.robots))]
        computed_moved = [0 for _ in range(len(self.robots))]
        final_positions = [None] * len(self.robots)
        occupied = {}  # Dictionary to record occupied cells.
        while True:
            updated = False
            for i in range(len(self.robots)):
            
                if computed_moved[i] != 0: 
                    continue

                pos = self.robots[i].position
                new_pos = proposed_positions[i]
                can_move = False
                if new_pos not in old_pos:
                    can_move = True
                else:
                    j = old_pos[new_pos]
                    if (j != i) and (computed_moved[j] == 0): # We must wait for the conflict resolve
                        continue
                    # We can decide where the robot can go now
                    can_move = True

                if can_move:
                    # print("Updated: ", i, new_pos)
                    if new_pos not in occupied:
                        occupied[new_pos] = i
                        final_positions[i] = new_pos
                        computed_moved[i] = 1
                        moved_robots[i] = 1
                        updated = True
                    else:
                        new_pos = pos
                        occupied[new_pos] = i
                        final_positions[i] = pos
                        computed_moved[i] = 1
                        moved_robots[i] = 0
                        updated = True

                if updated:
                    break

            if not updated:
                break
        #print("Computed postions: ", final_positions)
        for i in range(len(self.robots)):
            if computed_moved[i] == 0:
                final_positions[i] = self.robots[i].position 
        
        # Update robot positions and apply movement cost when applicable.
        for i, robot in enumerate(self.robots):
            move, pkg_act = actions[i]
            if move in ['L', 'R', 'U', 'D'] and final_positions[i] != robot.position:
                r += self.move_cost
            robot.position = final_positions[i]

        # -------- Process Package Actions --------
        for i, robot in enumerate(self.robots):
            move, pkg_act = actions[i]
            #print(i, move, pkg_act)
            # Pick up action.
            if pkg_act == '1':
                if robot.carrying == 0:
                    # Check for available packages at the current cell.
                    for j in range(len(self.packages)):
                        if self.packages[j].status == 'waiting' and self.packages[j].start == robot.position and self.packages[j].start_time <= self.t:
                            # Pick the package with the smallest package_id.
                            package_id = self.packages[j].package_id
                            robot.carrying = package_id
                            self.packages[j].status = 'in_transit'
                            # print(package_id, 'in transit')
                            break

            # Drop action.
            elif pkg_act == '2':
                if robot.carrying != 0:
                    package_id = robot.carrying
                    target = self.packages[package_id - 1].target
                    # Check if the robot is at the target position.
                    if robot.position == target:
                        # Update package status to delivered.
                        pkg = self.packages[package_id - 1]
                        pkg.status = 'delivered'
                        # Apply reward based on whether the delivery is on time.
                        if self.t <= pkg.deadline:
                            r += self.delivery_reward
                        else:
                            # Example: a reduced reward for late delivery.
                            r += self.delay_reward
                        robot.carrying = 0  
        
        # Increment the simulation timestep.
        self.t += 1

        self.total_reward += r

        done = False
        infos = {}
        if self.check_terminate():
            done = True
            infos['total_reward'] = self.total_reward
            infos['total_time_steps'] = self.t

        return self.get_state(), r, done, infos
    
    def check_terminate(self):
        if self.t == self.max_time_steps:
            return True
        
        for p in self.packages:
            if p.status != 'delivered':
                return False
            
        return True

    def compute_new_position(self, position, move):
        """
        Computes the intended new position for a robot given its current position and move command.
        """
        r, c = position
        if move == 'S':
            return (r, c)
        elif move == 'L':
            return (r, c - 1)
        elif move == 'R':
            return (r, c + 1)
        elif move == 'U':
            return (r - 1, c)
        elif move == 'D':
            return (r + 1, c)
        else:
            return (r, c)

    def valid_position(self, pos):
        """
        Checks if the new position is within the grid and not an obstacle.
        """
        r, c = pos
        if r < 0 or r >= self.n_rows or c < 0 or c >= self.n_cols:
            return False
        if self.grid[r][c] == 1:
            return False
        return True

    def render(self):
        """
        A simple text-based rendering of the map showing obstacles and robot positions.
        Obstacles are represented by 1, free cells by 0, and robots by 'R'.
        """
        # Make a deep copy of the grid
        grid_copy = [row[:] for row in self.grid]
        for i, robot in enumerate(self.robots):
            r, c = robot.position
            grid_copy[r][c] = 'R%i'%i
        for row in grid_copy:
            print('\t'.join(str(cell) for cell in row))

### Package storage

In [3]:
"""
{'package_id' : 'x',
                    'y',
                        'target_x',
                            'target_y',
                                'start_time',
                                    'deadline',
                                        'status'
}
"""
package_dict = {}

In [4]:
def state_to_agent(package_dict, state):
    if state['time_step'] == 0:
        package_dict = {}
    packages_in_carry = set()
    if len(state['packages']) > 0:
        for package in state['packages']:
            package_id = package[0]
            x, y, target_x, target_y, start_time, deadline = package[1:]
            status = 'waiting'
            package_dict[package_id] = {
                'x': x,
                'y': y,
                'target_x': target_x,
                'target_y': target_y,
                'start_time': start_time,
                'deadline': deadline,
                'status': status
            }
    for robot in state['robots']:
        package_carry = robot[2]
        if package_carry != 0:
            package_dict[package_carry]['status'] = 'carrying'
            packages_in_carry.add(package_carry)
    for package_id in package_dict:
        if package_dict[package_id]['status'] == 'carrying' and package_id not in packages_in_carry:
            package_dict[package_id]['status'] = 'Delivered'
    state = {
        'robots': state['robots'],
        'packages':[(package_id,
                        package_dict[package_id]['x'],
                        package_dict[package_id]['y'],
                            package_dict[package_id]['target_x'],
                            package_dict[package_id]['target_y'],
                                package_dict[package_id]['start_time'],
                                package_dict[package_id]['deadline'],
                                    package_dict[package_id]['status'])for package_id in package_dict],
        'map': state['map'],
        'time_step': state['time_step'],
    }
    return state, package_dict

### Critic observation

In [5]:
def convert_global_state_to_critic_input(global_state_dict: dict, all_packages_data: dict, map_dims: int = 20, max_time_steps: int = None):
    """
    Chuyển đổi global state và thông tin package đầy đủ thành input tensor cho Critic.

    Args:
        global_state_dict (dict): Output từ state to agent.
        all_packages_data (dict): Dictionary chứa thông tin đầy đủ của tất cả các package
                                  (ví dụ: agents_object.packages).
                                  Format: {pkg_id: {'x': int, 'y': int, 'target_x': int,
                                                    'target_y': int, 'deadline': int,
                                                    'status': str, 'start_time': int}}
        map_dims (tuple): (MAP_HEIGHT, MAP_WIDTH).
        max_time_steps (int, optional): Tổng số bước thời gian tối đa trong một episode.
                                        Cần thiết nếu use_time_channel=True.
        use_time_channel (bool): True nếu muốn thêm kênh thời gian còn lại.

    Returns:
        np.ndarray: Input tensor cho Critic.
    """
    MAP_HEIGHT, MAP_WIDTH = map_dims
    num_channels = 7

    critic_input = np.zeros((num_channels, MAP_HEIGHT, MAP_WIDTH), dtype=np.float32)
    current_time = global_state_dict['time_step']

    # --- Channel 0: Map Obstacles ---
    grid_map = np.array(global_state_dict['map'])
    critic_input[0, :, :] = grid_map[:MAP_HEIGHT, :MAP_WIDTH]

    # --- Lấy thông tin robots ---
    robots_info = global_state_dict['robots']  # list of (x, y, carrying_package_id)

    # --- Channel 1: Vị trí của tất cả Robots ---
    # --- Channel 2: Trạng thái mang hàng của Robots ---
    for r_data in robots_info:
        # Tọa độ từ state là 1-based, chuyển về 0-based cho array
        r_x_1based, r_y_1based, carrying_package_id = r_data
        r_row, r_col = r_x_1based - 1, r_y_1based - 1

        if 0 <= r_row < MAP_HEIGHT and 0 <= r_col < MAP_WIDTH:
            critic_input[1, r_row, r_col] = 1.0  # Đánh dấu vị trí robot
            if carrying_package_id != 0:
                critic_input[2, r_row, r_col] = 1.0  # Đánh dấu robot đang mang hàng


    for pkg_id, pkg_details in all_packages_data.items():
        # Tọa độ từ package_details là 1-based
        p_x_1based, p_y_1based = pkg_details['x'], pkg_details['y']
        pt_x_1based, pt_y_1based = pkg_details['target_x'], pkg_details['target_y']

        p_row, p_col = p_x_1based - 1, p_y_1based - 1
        pt_row, pt_col = pt_x_1based - 1, pt_y_1based - 1

        status = pkg_details['status']
        deadline = pkg_details['deadline']

        # Tính toán urgency (giống như trong state_to_agent của bạn nhưng có chuẩn hóa)
        time_to_deadline = max(0, deadline - current_time) # Tránh số âm nếu đã qua deadline
        # epsilon để tránh chia cho 0, và một giá trị cơ sở nhỏ để urgency không quá lớn
        urgency_val = 1.0 / (time_to_deadline + 1e-5 + 0.1) # Thêm 0.1 để giảm độ lớn của urgency

        normalized_urgency = min(1.0, urgency_val / 10.0) # Giả sử max urgency sau khi +0.1 là khoảng 10

        if status == 'waiting':
            # --- Channel 3: Vị trí các Package đang chờ (Waiting) ---
            if 0 <= p_row < MAP_HEIGHT and 0 <= p_col < MAP_WIDTH:
                critic_input[3, p_row, p_col] = 1.0

            # --- Channel 4: Vị trí đích của các Package đang chờ (Waiting) ---
            if 0 <= pt_row < MAP_WIDTH and 0 <= pt_col < MAP_WIDTH: # Sửa: pt_row < MAP_HEIGHT
                 critic_input[4, pt_row, pt_col] = 1.0

            # --- Channel 6: Mức độ khẩn cấp (đặt tại vị trí đích) ---
            if 0 <= pt_row < MAP_HEIGHT and 0 <= pt_col < MAP_WIDTH:
                critic_input[6, pt_row, pt_col] = max(critic_input[6, pt_row, pt_col], normalized_urgency)

        elif status == 'carrying': # Hoặc 'in_transit' tùy theo định nghĩa của bạn
            # Package 'carrying' không được đánh dấu ở Channel 3 hoặc 4
            # Vị trí hiện tại của nó là vị trí robot (đã có ở Channel 1 & 2)

            # --- Channel 5: Vị trí đích của các Package đang vận chuyển (Carrying) ---
            if 0 <= pt_row < MAP_HEIGHT and 0 <= pt_col < MAP_WIDTH:
                critic_input[5, pt_row, pt_col] = 1.0

            # --- Channel 6: Mức độ khẩn cấp (đặt tại vị trí đích) ---
            if 0 <= pt_row < MAP_HEIGHT and 0 <= pt_col < MAP_WIDTH:
                critic_input[6, pt_row, pt_col] = max(critic_input[6, pt_row, pt_col], normalized_urgency)
    return critic_input

### Actor observation

In [6]:
def convert_state_to_actor_input(global_state_dict, all_packages_data, current_agent_idx, map_dims):

    MAP_HEIGHT, MAP_WIDTH = map_dims
    num_actor_channels = 6 # Theo đề xuất 6 kênh ở trên
    actor_obs = np.zeros((num_actor_channels, MAP_HEIGHT, MAP_WIDTH), dtype=np.float32)
    current_time = global_state_dict['time_step']

    # --- Channel 0: Map Obstacles ---
    grid_map = np.array(global_state_dict['map'])
    actor_obs[0, :, :] = grid_map[:MAP_HEIGHT, :MAP_WIDTH]

    # --- Lấy thông tin robots ---
    robots_info = global_state_dict['robots']  # list of (x, y, carrying_package_id)

    # Dữ liệu của agent hiện tại
    agent_data = robots_info[current_agent_idx]
    agent_x_1based, agent_y_1based, agent_carrying_pkg_id = agent_data
    agent_r, agent_c = agent_x_1based - 1, agent_y_1based - 1

    # --- Channel 1: Vị trí của Agent hiện tại ---
    if 0 <= agent_r < MAP_HEIGHT and 0 <= agent_c < MAP_WIDTH:
        actor_obs[1, agent_r, agent_c] = 1.0

    # --- Channel 2: Vị trí của các Agent khác ---
    for i, other_robot_data in enumerate(robots_info):
        if i == current_agent_idx:
            continue # Bỏ qua chính agent này

        other_x_1based, other_y_1based, _ = other_robot_data
        other_r, other_c = other_x_1based - 1, other_y_1based - 1
        if 0 <= other_r < MAP_HEIGHT and 0 <= other_c < MAP_WIDTH:
            actor_obs[2, other_r, other_c] = 1.0

    # --- Xử lý Packages ---

    # Chuẩn hóa urgency (có thể dùng hàm riêng nếu logic phức tạp)
    def get_normalized_urgency(pkg_deadline, current_time_step):
        time_to_deadline = max(0, pkg_deadline - current_time_step)
        urgency_val = 1.0 / (time_to_deadline + 1e-5 + 0.1) # Giống critic
        # Ví dụ chuẩn hóa: urgency_val / (urgency_val + K), K=4
        normalized = urgency_val / (urgency_val + 4.0)
        return min(1.0, max(0.0, normalized)) # Đảm bảo trong [0,1]

    # Xử lý package mà agent hiện tại đang mang
    if agent_carrying_pkg_id != 0:
        if agent_carrying_pkg_id in all_packages_data:
            carried_pkg_details = all_packages_data[agent_carrying_pkg_id]

            # --- Channel 4: Vị trí đích của Package mà Agent hiện tại đang mang ---
            pt_x_1based = carried_pkg_details['target_x']
            pt_y_1based = carried_pkg_details['target_y']
            pt_r, pt_c = pt_x_1based - 1, pt_y_1based - 1
            if 0 <= pt_r < MAP_HEIGHT and 0 <= pt_c < MAP_WIDTH:
                actor_obs[4, pt_r, pt_c] = 1.0

            # --- Channel 5: Mức độ khẩn cấp (Cách 2 - Agent-specific) ---
            urgency = get_normalized_urgency(carried_pkg_details['deadline'], current_time)
            if 0 <= agent_r < MAP_HEIGHT and 0 <= agent_c < MAP_WIDTH:
                 actor_obs[5, agent_r, agent_c] = max(actor_obs[5, agent_r, agent_c], urgency)


    # Xử lý các package khác (chủ yếu là 'waiting' cho agent không mang hàng)
    for pkg_id, pkg_details in all_packages_data.items():
        status = pkg_details['status']

        if status == 'waiting':
            # --- Channel 3: Vị trí các Package đang chờ (Waiting) ---
            p_x_1based, p_y_1based = pkg_details['x'], pkg_details['y']
            p_r, p_c = p_x_1based - 1, p_y_1based - 1
            if 0 <= p_r < MAP_HEIGHT and 0 <= p_c < MAP_WIDTH:
                actor_obs[3, p_r, p_c] = 1.0

            # --- Channel 5: Mức độ khẩn cấp (Cách 2 - Agent-specific) ---
            # Nếu agent không mang hàng, đặt urgency của package 'waiting' tại vị trí HIỆN TẠI của package đó
            # (để agent biết package nào gần/khẩn cấp để nhặt)
            if agent_carrying_pkg_id == 0: # Chỉ xem xét nếu agent đang rảnh
                urgency = get_normalized_urgency(pkg_details['deadline'], current_time)
                if 0 <= p_r < MAP_HEIGHT and 0 <= p_c < MAP_WIDTH:
                    actor_obs[5, p_r, p_c] = max(actor_obs[5, p_r, p_c], urgency)
    return actor_obs

### Critic Network

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class CriticCNN_V3(nn.Module):
    def __init__(self, input_channels=7):
        super(CriticCNN_V3, self).__init__()
        self.input_channels = input_channels

        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        # Thêm lớp conv4
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1) # Hoặc giữ 128 filters
        self.bn4 = nn.BatchNorm2d(256)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc1 = nn.Linear(256, 128) # Input từ conv4 (256 filters)
        self.fc_critic_head = nn.Linear(128, 1)

    def forward(self, global_observation):
        x = F.relu(self.bn1(self.conv1(global_observation)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x))) # Thêm forward qua lớp mới
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        state_value = self.fc_critic_head(x)
        return state_value

### Actor Network

In [8]:
class ActorCNN(nn.Module):
    def __init__(self, input_channels=6, num_actions=15):
        """
        Actor Network sử dụng CNN để xử lý local observation và đưa ra logit hành động.

        Args:
            input_channels (int): Số lượng kênh của local observation (ví dụ: 6).
            num_actions (int): Tổng số hành động mà agent có thể thực hiện (ví dụ: 5 di chuyển * 3 package_ops = 15).
        """
        super(ActorCNN, self).__init__()
        self.input_channels = input_channels
        self.num_actions = num_actions

        # Convolutional layers (Kiến trúc ví dụ, có thể điều chỉnh)
        # Input: (batch_size, input_channels, MAP_HEIGHT, MAP_WIDTH)
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.AdaptiveAvgPool2d((1, 1)) # Global Average Pooling

        # Fully connected layers
        # Kích thước sau pooling là (batch_size, 128, 1, 1) -> flatten thành (batch_size, 128)
        self.fc1 = nn.Linear(128, 128) # Giữ nguyên 128 hoặc giảm xuống
        self.fc_actor_head = nn.Linear(128, num_actions) # Output logits cho các hành động

    def forward(self, local_observation):
        """
        Forward pass của Actor network.

        Args:
            local_observation (torch.Tensor): Local observation của agent,
                                             shape (batch_size, input_channels, height, width).

        Returns:
            torch.Tensor: Logits cho các hành động, shape (batch_size, num_actions).
        """
        # CNN feature extraction
        x = F.relu(self.bn1(self.conv1(local_observation)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(x) # (batch_size, 128, 1, 1)

        # Flatten
        x = x.view(x.size(0), -1) # (batch_size, 128)

        # Fully connected layers
        x = F.relu(self.fc1(x))
        action_logits = self.fc_actor_head(x) # (batch_size, num_actions)

        return action_logits

    @staticmethod
    def decode_action(action_index):
        """
        Chuyển đổi một action index (0-14) thành (move_action_str, package_action_str).
        Ví dụ: 0 -> ('S', '0')
        """
        if not (0 <= action_index < 15):
            raise ValueError("Action index phải nằm trong khoảng [0, 14]")

        move_actions = ['S', 'L', 'R', 'U', 'D']  # Stay, Left, Right, Up, Down
        package_actions = ['0', '1', '2']          # Do_nothing, Pickup, Drop

        move_idx = action_index // len(package_actions) # 0, 1, 2, 3, 4
        package_idx = action_index % len(package_actions) # 0, 1, 2

        return move_actions[move_idx], package_actions[package_idx]

    @staticmethod
    def encode_action(move_action_str, package_action_str):
        """
        Chuyển đổi (move_action_str, package_action_str) thành action index (0-14).
        Ví dụ: ('S', '0') -> 0
        """
        move_actions = ['S', 'L', 'R', 'U', 'D']
        package_actions = ['0', '1', '2']

        try:
            move_idx = move_actions.index(move_action_str)
            package_idx = package_actions.index(package_action_str)
        except ValueError:
            raise ValueError(f"Hành động không hợp lệ: ({move_action_str}, {package_action_str})")

        return move_idx * len(package_actions) + package_idx

In [9]:
class MAPPORolloutBuffer:
    """
    Rollout Buffer for Centralized Critic / Decentralized Actor (CC/DA) MAPPO.
    Stores experience for a fixed-size rollout and provides minibatches for training.
    Assumes per-agent rewards and a single value estimate from the Centralized Critic
    for the global state.

    Designed to store data matching the output of your environment and observation
    conversion functions (convert_global_state_to_critic_input,
    convert_state_to_actor_input, compute_shaped_rewards).
    """
    def __init__(self, buffer_size: int, num_agents: int, map_dims: tuple[int, int],
                 critic_obs_channels: int, actor_obs_channels: int,
                 gamma: float = 0.99, gae_lambda: float = 0.95, device: str | torch.device = 'cpu'):
        """
        Initializes the MAPPO Rollout Buffer.

        Args:
            buffer_size (int): The maximum number of environment steps to store
                               in the buffer (rollout length per training epoch).
            num_agents (int): The number of agents (robots) in the environment.
            map_dims (tuple): The dimensions of the map (MAP_HEIGHT, MAP_WIDTH).
            critic_obs_channels (int): Number of channels in the global observation
                                       for the Critic (based on convert_global_state_to_critic_input).
            actor_obs_channels (int): Number of channels in the local observation
                                      for each Actor (based on convert_state_to_actor_input).
            gamma (float): Discount factor for rewards.
            gae_lambda (float): Lambda parameter for Generalized Advantage Estimation (GAE).
            device (str or torch.device): The device ('cpu' or 'cuda') to move tensors to
                                          when sampling for training.
        """
        self.buffer_size = buffer_size
        self.num_agents = num_agents
        self.map_height, self.map_width = map_dims
        self.critic_obs_channels = critic_obs_channels
        self.actor_obs_channels = actor_obs_channels
        self.gamma = gamma
        self.gae_lambda = gae_lambda
        self.device = device

        self._allocate_storage()
        self.reset()

    def _allocate_storage(self):
        """
        Allocates NumPy arrays for storing the rollout data.
        Each array is sized to hold `buffer_size` steps.
        """
        # Global state observation for the Centralized Critic at time step t
        # Shape: (buffer_size, C_critic, H, W)
        self.global_states = np.zeros(
            (self.buffer_size, self.critic_obs_channels, self.map_height, self.map_width),
            dtype=np.float32
        )

        # Local state observation for *each* Decentralized Actor at time step t
        # Shape: (buffer_size, num_agents, C_actor, H, W)
        self.agent_local_states = np.zeros(
            (self.buffer_size, self.num_agents, self.actor_obs_channels, self.map_height, self.map_width),
            dtype=np.float32
        )

        # Action taken by each Agent at time step t (stored as integer index)
        # Shape: (buffer_size, num_agents)
        self.actions = np.zeros(
            (self.buffer_size, self.num_agents),
            dtype=np.int64
        )

        # Shaped Reward received by *each* Agent after taking action at time step t
        # Shape: (buffer_size, num_agents)
        self.rewards = np.zeros(
            (self.buffer_size, self.num_agents),
            dtype=np.float32
        )

        # Terminal flag indicating if the episode ended after time step t
        # Shape: (buffer_size,)
        self.dones = np.zeros(
            self.buffer_size,
            dtype=np.bool_
        )

        # Log probability of the action taken by each Agent at time step t
        # (from the policy used to select the action)
        # Shape: (buffer_size, num_agents)
        self.log_probs = np.zeros(
            (self.buffer_size, self.num_agents),
            dtype=np.float32
        )

        # Value estimate V(s_t) from the Centralized Critic for the global state at time step t
        # Shape: (buffer_size,) - storing a single scalar value per step
        self.values = np.zeros(
            self.buffer_size,
            dtype=np.float32
        )

        # Placeholders for Advantages and Returns (calculated AFTER data collection)
        # These are calculated PER AGENT based on the global value estimate and per-agent rewards.
        # Shape: (buffer_size, num_agents)
        self.advantages = np.zeros(
            (self.buffer_size, self.num_agents),
            dtype=np.float32
        )
        self.returns = np.zeros(
            (self.buffer_size, self.num_agents),
            dtype=np.float32
        )

    def reset(self):
        """
        Resets the buffer pointers and effectively clears the collected data
        for the next rollout. The underlying NumPy arrays are kept to avoid
        reallocation overhead, but the `ptr` indicates the currently active data range.
        """
        self.ptr = 0  # Pointer to the next empty slot to add data
        self.path_start_idx = 0  # Index marking the start of the current episode segment

    def add(self, global_obs: np.ndarray, agent_obs_list: list[np.ndarray],
            action_list: list[int], reward_list: list[float], done: bool,
            log_prob_list: list[float], value: float):
        """
        Adds one step of experience (from t to t+1) to the buffer.

        Args:
            global_obs (np.ndarray): Global observation for the Critic at time t.
                                     Shape (C_critic, H, W).
            agent_obs_list (list[np.ndarray]): List of local observations for each agent
                                             at time t. Each np.ndarray shape (C_actor, H, W).
            action_list (list[int]): List of action indices taken by each agent at time t.
                                     Length num_agents.
            reward_list (list[float]): List of shaped rewards received by each agent
                                       after action at time t. Length num_agents.
            done (bool): True if the episode terminates after time step t.
            log_prob_list (list[float]): List of log probabilities for the actions taken
                                         at time t. Length num_agents.
            value (float): Value estimate V(s_t) from the Centralized Critic for global_obs.
        """
        if self.ptr >= self.buffer_size:
            # In a standard rollout buffer, this signals the end of data collection for the epoch.
            # No more data is added until after training and reset.
            # print("Warning: Buffer is full. Cannot add more steps.") # Optional warning
            return

        # Store data at the current pointer position
        self.global_states[self.ptr] = global_obs
        # Stack the list of agent observations into a single numpy array
        self.agent_local_states[self.ptr] = np.stack(agent_obs_list, axis=0) # Shape becomes (num_agents, C_actor, H, W)
        self.actions[self.ptr] = np.array(action_list, dtype=np.int64)
        self.rewards[self.ptr] = np.array(reward_list, dtype=np.float32)
        self.dones[self.ptr] = done
        self.log_probs[self.ptr] = np.array(log_prob_list, dtype=np.float32)
        self.values[self.ptr] = value # Store scalar V(s_t)

        # If the episode finished at this step, calculate GAE/Returns for the completed segment
        if done:
            # Calculate GAE and Returns for the segment from path_start_idx up to (and including) ptr
            # The last_value for a terminal state is 0.0.
            self._compute_returns_and_advantages_segment(
                path_start_idx=self.path_start_idx,
                path_end_idx=self.ptr + 1, # Segment ends *after* the step at ptr
                last_value=0.0 # V(s_{T+1}) = 0 for a terminal state s_T
            )
            # Start index for the next segment/episode is the step after the current one
            self.path_start_idx = self.ptr + 1

        # Increment the pointer for the next `add` call
        self.ptr += 1

    def _compute_returns_and_advantages_segment(self, path_start_idx: int, path_end_idx: int, last_value: float):
        """
        Internal helper to compute Returns and Generalized Advantage Estimation (GAE)
        for a specific segment of the buffer ([path_start_idx, path_end_idx - 1]).

        Args:
            path_start_idx (int): The starting index (inclusive) in the buffer.
            path_end_idx (int): The ending index (exclusive) in the buffer.
                                This is one step BEYOND the last step in the segment.
            last_value (float): The estimated value of the state *after* the last
                                step in this segment (V(s_T) if T is the last step
                                in the segment). Used for bootstrapping the returns/GAE.
        """
        # Extract data for the current segment
        rewards_segment = self.rewards[path_start_idx : path_end_idx] # Shape: (segment_len, num_agents)
        values_segment = self.values[path_start_idx : path_end_idx]   # Shape: (segment_len,)
        dones_segment = self.dones[path_start_idx : path_end_idx]     # Shape: (segment_len,)

        segment_len = rewards_segment.shape[0]

        # Initialize GAE for the current segment. It represents the advantage
        # *from* the state *after* the segment ends.
        current_gae = np.zeros(self.num_agents, dtype=np.float32)

        # Iterate backwards through the segment to calculate GAE and Returns
        # t is the index within the segment [0, segment_len - 1]
        # buffer_t is the corresponding index in the full buffer [path_start_idx, path_end_idx - 1]
        for t in reversed(range(segment_len)):
            buffer_t = path_start_idx + t

            # Determine V(s_{t+1}) and the non-terminal factor for the next state
            if t == segment_len - 1:
                # This is the last step of the segment. The "next state" is after the segment.
                # Its value is last_value, and its non-terminal factor depends on whether
                # the state at 'buffer_t' was terminal (dones_segment[t]).
                next_non_terminal = 1.0 - dones_segment[t]
                next_value = last_value
            else:
                # Not the last step of the segment. The "next state" is at buffer_t + 1.
                # Its value is values_segment[t+1], and its non-terminal factor depends on
                # whether the state at 'buffer_t + 1' was terminal (dones_segment[t+1]).
                next_non_terminal = 1.0 - dones_segment[t+1]
                next_value = values_segment[t+1] # Scalar value

            # delta_t (Temporal Difference error) for each agent:
            # delta_t = r_t + gamma * V(s_{t+1}) * (1 - done_{t+1}) - V(s_t)
            # rewards_segment[t] is (num_agents,), values_segment[t] is scalar.
            # Broadcasting works correctly.
            delta = rewards_segment[t] + self.gamma * next_value * next_non_terminal - values_segment[t]

            # GAE_t (Generalized Advantage Estimation) for each agent:
            # GAE_t = delta_t + gamma * lambda * GAE_{t+1} * (1 - done_{t+1})
            # current_gae holds GAE_{t+1} from the previous iteration.
            current_gae = delta + self.gamma * self.gae_lambda * current_gae * next_non_terminal

            # Store the calculated GAE for the step at buffer_t
            self.advantages[buffer_t] = current_gae

        # Returns_t = Advantages_t + Values_t
        # Returns are simply GAE + the value prediction V(s_t)
        # values_segment is (segment_len,), self.advantages[...] is (segment_len, num_agents)
        # Need to add a new axis to values_segment for correct broadcasting before addition
        self.returns[path_start_idx : path_end_idx] = self.advantages[path_start_idx : path_end_idx] + values_segment[:, np.newaxis]


    def compute_final_returns_and_advantages(self, last_global_state_obs: np.ndarray | None, last_value_fn: callable):
        """
        Computes the Returns and Advantages for the final segment of the buffer
        after all steps have been added. This method should be called once per rollout
        after the main data collection loop finishes (either buffer full or episode done).

        Args:
            last_global_state_obs (np.ndarray or None): The global observation of the
                                                       state *after* the last step
                                                       added to the buffer (s_T), but
                                                       *only if* the episode did NOT
                                                       terminate at the last step (dones[ptr-1] is False).
                                                       Shape (C_critic, H, W). Is None if the
                                                       last step added had done=True.
            last_value_fn (callable): A function (typically the Critic's forward method)
                                      that takes a global observation tensor and returns
                                      the value estimate tensor. Used to get V(s_T) if the
                                      rollout ends mid-episode (`last_global_state_obs` is not None).
                                      Should accept a batch (e.g., (1, C, H, W)).
        """
        # If the buffer is empty, there's nothing to compute
        if self.ptr == 0:
            return

        # Determine the value of the state *after* the last step added to the buffer (s_T).
        # This value is used to bootstrap the GAE/Returns calculation for the final segment.
        if self.dones[self.ptr - 1]:
            # The last state added (s_{ptr-1}) was terminal, so the value of the next state (s_T) is 0.
            final_bootstrap_value = 0.0
        else:
            # The rollout ended mid-episode (buffer filled or MAX_TIME_STEPS reached).
            # We need to predict the value of the state *after* the last step added (s_T) using the Critic.
            if last_global_state_obs is None:
                 raise ValueError("last_global_state_obs must be provided to compute the final bootstrap value when the rollout doesn't end with done=True.")

            # Convert the last global state observation to a tensor and predict its value using the Critic
            last_obs_tensor = torch.tensor(last_global_state_obs[np.newaxis, ...], dtype=torch.float32).to(self.device)
            with torch.no_grad():
                # Assuming the critic model takes a batch and returns a batch of scalar values
                final_bootstrap_value = last_value_fn(last_obs_tensor).item() # .item() extracts scalar from 1-element tensor

        # Compute GAE and Returns for the final segment.
        # This segment includes all steps from the last episode's start index (path_start_idx)
        # up to the last step added to the buffer (ptr - 1).
        self._compute_returns_and_advantages_segment(
            path_start_idx=self.path_start_idx, # Start of the segment
            path_end_idx=self.ptr,              # End of the segment (exclusive)
            last_value=final_bootstrap_value    # Bootstrap value for the state after this segment
        )


    def sample_minibatches(self, num_minibatches: int, normalize_advantages: bool = True):
        """
        Generates minibatches from the collected rollout data.
        This method should only be called AFTER compute_final_returns_and_advantages
        has been called to ensure advantages and returns are computed.

        Data is flattened such that each (timestep, agent) pair becomes a single
        sample, which is then used for random minibatching.

        Args:
            num_minibatches (int): The number of minibatches to divide the collected
                                   data into. The size of each minibatch will be
                                   (total_steps * num_agents) // num_minibatches.
            normalize_advantages (bool): Whether to normalize advantages across
                                         all collected samples. Standard practice in PPO.

        Yields:
            tuple: A minibatch of tensors on the specified device, containing:
                   (global_states_batch, agent_local_states_batch, actions_batch,
                    old_log_probs_batch, old_values_batch, returns_batch, advantages_batch)
                   Each tensor has shape (minibatch_size, ...).
        """
        # Total number of valid steps collected in this rollout
        num_steps_collected = self.ptr

        if num_steps_collected == 0:
            print("Warning: Buffer is empty, cannot sample minibatches.")
            return # Nothing to yield

        # Flatten the data across time steps and agents
        # This treats each (timestep, agent) combination as a single sample for batching
        total_samples = num_steps_collected * self.num_agents

        # Reshape global state and value to match the size of agent data
        # (buffer_size, C, H, W) -> (buffer_size * num_agents, C, H, W) by repeating per agent
        flat_global_states = np.repeat(self.global_states[:num_steps_collected, np.newaxis, ...],
                                       self.num_agents, axis=1).reshape(-1, self.critic_obs_channels, self.map_height, self.map_width)

        # (buffer_size, num_agents, C_actor, H, W) -> (buffer_size * num_agents, C_actor, H, W)
        flat_agent_local_states = self.agent_local_states[:num_steps_collected].reshape(-1, self.actor_obs_channels, self.map_height, self.map_width)

        # (buffer_size, num_agents) -> (buffer_size * num_agents,)
        flat_actions = self.actions[:num_steps_collected].reshape(-1)
        flat_log_probs = self.log_probs[:num_steps_collected].reshape(-1)
        flat_returns = self.returns[:num_steps_collected].reshape(-1)
        flat_advantages = self.advantages[:num_steps_collected].reshape(-1)

        # (buffer_size,) -> (buffer_size * num_agents,) by repeating the scalar value per agent
        flat_values = np.repeat(self.values[:num_steps_collected, np.newaxis],
                                self.num_agents, axis=1).reshape(-1)

        # Convert NumPy arrays to PyTorch Tensors and move to device
        flat_global_states_tensor = torch.tensor(flat_global_states, dtype=torch.float32).to(self.device)
        flat_agent_local_states_tensor = torch.tensor(flat_agent_local_states, dtype=torch.float32).to(self.device)
        flat_actions_tensor = torch.tensor(flat_actions, dtype=torch.int64).to(self.device)
        flat_log_probs_tensor = torch.tensor(flat_log_probs, dtype=torch.float32).to(self.device)
        flat_values_tensor = torch.tensor(flat_values, dtype=torch.float32).to(self.device)
        flat_returns_tensor = torch.tensor(flat_returns, dtype=torch.float32).to(self.device)
        flat_advantages_tensor = torch.tensor(flat_advantages, dtype=torch.float32).to(self.device)

        # Normalize advantages if requested
        if normalize_advantages:
            # Add a small epsilon to avoid division by zero if std is 0 (highly unlikely with actual data)
            advantages_mean = flat_advantages_tensor.mean()
            advantages_std = flat_advantages_tensor.std()
            flat_advantages_tensor = (flat_advantages_tensor - advantages_mean) / (advantages_std + 1e-8)

        # Create a sampler to yield indices for minibatches
        # We sample random indices from the flattened range [0, total_samples - 1]
        minibatch_size = total_samples // num_minibatches
        if minibatch_size == 0:
             raise ValueError(f"Minibatch size is 0. Increase buffer_size or decrease num_minibatches. total_samples={total_samples}, num_minibatches={num_minibatches}")

        sampler = BatchSampler(
            SubsetRandomSampler(range(total_samples)), # Samples indices randomly
            minibatch_size,                           # Size of each batch of indices
            drop_last=True                            # Drop the last batch if it's smaller than minibatch_size
        )

        # Yield minibatches using the sampled indices
        for indices in sampler:
            yield (
                flat_global_states_tensor[indices],
                flat_agent_local_states_tensor[indices],
                flat_actions_tensor[indices],
                flat_log_probs_tensor[indices],
                flat_values_tensor[indices],
                flat_returns_tensor[indices],
                flat_advantages_tensor[indices]
            )

    def ready_to_sample(self):
        """
        Checks if the buffer has collected enough steps to form a full rollout.
        (i.e., reached the buffer_size).
        """
        return self.ptr >= self.buffer_size

### Trainning set up



In [10]:
import collections

def run_bfs_distance(grid_map, start, goal):
    """
    Runs BFS starting from the goal position to find the shortest distance
    to the start position on a grid map.

    The BFS calculates the shortest distance from the goal to every reachable cell.
    The function returns the shortest distance from the start to the goal.

    :param grid_map: 2D list representing the map (0: free, 1: obstacle).
                     Assumes grid_map is rectangular and non-empty.
    :param start: Tuple (row, column) of the starting position (0-indexed).
    :param goal: Tuple (row, column) of the goal position (0-indexed).
    :return: The shortest distance (an integer) from 'start' to 'goal'.
             Returns float('inf') if start is the goal, if start or goal
             is an obstacle, or if the goal is unreachable from the start.
             Returns float('inf') for invalid start/goal positions outside map bounds.
    """
    n_rows = len(grid_map)
    n_cols = len(grid_map[0]) if n_rows > 0 else 0

    # Basic validation: Check if start and goal are within bounds and not obstacles
    if not (0 <= start[0] < n_rows and 0 <= start[1] < n_cols) or \
       not (0 <= goal[0] < n_rows and 0 <= goal[1] < n_cols):
        # Positions are out of bounds
        return float('inf')

    if grid_map[start[0]][start[1]] == 1 or grid_map[goal[0]][goal[1]] == 1:
         # Start or goal position is an obstacle
         return float('inf')

    # Edge case: Start is already the goal
    if start == goal:
        return 0 # Distance is 0

    # Initialize BFS structures
    # queue stores positions to visit
    queue = collections.deque()
    # distance dictionary stores the shortest distance from the goal to each position
    distance = {}

    # Define possible moves (dr, dc)
    moves = [(-1, 0), (1, 0), (0, -1), (0, 1)] # Up, Down, Left, Right

    # Start BFS from the goal position
    queue.append(goal)
    distance[goal] = 0

    # Run BFS
    while queue:
        current_pos = queue.popleft()
        dist = distance[current_pos]

        # Explore neighbors
        for dr, dc in moves:
            neighbor_pos = (current_pos[0] + dr, current_pos[1] + dc)

            # Check bounds
            if not (0 <= neighbor_pos[0] < n_rows and 0 <= neighbor_pos[1] < n_cols):
                continue

            # Check for obstacles
            if grid_map[neighbor_pos[0]][neighbor_pos[1]] == 1:
                continue

            # If neighbor has not been visited yet (distance is not set)
            if neighbor_pos not in distance:
                distance[neighbor_pos] = dist + 1
                queue.append(neighbor_pos)

    # After BFS, check if the start position was reached
    if start in distance:
        # Start is reachable from the goal, return the stored distance
        return distance[start]
    else:
        # Goal is unreachable from the start position
        return float('inf')

In [11]:
def compute_shaped_rewards(
    global_reward,
    prev_env_state_dict,
    current_env_state_dict,
    actions_taken_for_all_agents,
    persistent_packages_at_prev_state,
    num_agents,
):
    """
    Computes shaped rewards for each agent based on transitions and intended actions.
    Returns: tổng shaped reward (float), và shaped reward từng agent (np.array)
    """
    # --- Shaping Constants ---
    SHAPING_SUCCESSFUL_PICKUP = 4
    SHAPING_SUCCESSFUL_DELIVERY_ON_TIME = 200
    SHAPING_SUCCESSFUL_DELIVERY_LATE = 50
    SHAPING_MOVED_CLOSER_TO_TARGET = 1
    SHAPING_WASTED_PICKUP_ATTEMPT = 0
    SHAPING_WASTED_DROP_ATTEMPT = 0
    SHAPING_COLLISION_OR_STUCK = -0.5
    SHAPING_IDLE_WITH_AVAILABLE_TASKS = -0.5
    SHAPING_MOVED_AWAY_FROM_TARGET = -1


    shaped_rewards = np.zeros(num_agents, dtype=np.float32)
    current_time = int(current_env_state_dict['time_step'])


    for agent_idx in range(num_agents):
        prev_r, prev_c, prev_pkg = [int(x) for x in prev_env_state_dict['robots'][agent_idx]]
        curr_r, curr_c, curr_pkg = [int(x) for x in current_env_state_dict['robots'][agent_idx]]
        prev_r -= 1; prev_c -= 1; curr_r -= 1; curr_c -= 1
        move_str, pkg_op_str = actions_taken_for_all_agents[agent_idx]
        pkg_op = int(pkg_op_str)

        # 1. Nhặt/thả thành công
        if prev_pkg == 0 and curr_pkg != 0:
            shaped_rewards[agent_idx] += SHAPING_SUCCESSFUL_PICKUP
        elif prev_pkg != 0 and curr_pkg == 0:
            dropped_pkg = prev_pkg
            if dropped_pkg in persistent_packages_at_prev_state:
                pkg_info = persistent_packages_at_prev_state[dropped_pkg]
                if current_time <= pkg_info['deadline']:
                    shaped_rewards[agent_idx] += SHAPING_SUCCESSFUL_DELIVERY_ON_TIME
                else:
                    shaped_rewards[agent_idx] += SHAPING_SUCCESSFUL_DELIVERY_LATE

        # 2. Phạt hành động lãng phí
        if pkg_op == 1:  # Pick
            if prev_pkg != 0:
                shaped_rewards[agent_idx] += SHAPING_WASTED_PICKUP_ATTEMPT
            elif curr_pkg == 0:
                can_pickup = any(
                    persistent_packages_at_prev_state[pkg_id]['status'] == 'waiting' and
                    persistent_packages_at_prev_state[pkg_id]['start_time'] <= prev_env_state_dict['time_step'] and
                    (persistent_packages_at_prev_state[pkg_id]['x'],persistent_packages_at_prev_state[pkg_id]['y']) == (curr_r, curr_c)
                    for pkg_id in persistent_packages_at_prev_state
                )
                if not can_pickup:
                    shaped_rewards[agent_idx] += SHAPING_WASTED_PICKUP_ATTEMPT
        elif pkg_op == 2:  # Drop
            if prev_pkg == 0:
                shaped_rewards[agent_idx] += SHAPING_WASTED_DROP_ATTEMPT
            elif curr_pkg != 0:
                shaped_rewards[agent_idx] += 0

        # 3. Di chuyển
        moved = (prev_r, prev_c) != (curr_r, curr_c)
        intended_move = move_str != 'S'
        if intended_move and not moved:
            shaped_rewards[agent_idx] += SHAPING_COLLISION_OR_STUCK

        # Tính mục tiêu di chuyển
        target_pos = None
        if prev_pkg != 0 and prev_pkg in persistent_packages_at_prev_state:
            target_pos = (persistent_packages_at_prev_state[prev_pkg]['target_x'],
                          persistent_packages_at_prev_state[prev_pkg]['target_y'])
        else:
            # Gói waiting gần nhất
            waiting_pkgs = [
                pkg for pkg in persistent_packages_at_prev_state
                if persistent_packages_at_prev_state[pkg]['status'] == 'waiting' and persistent_packages_at_prev_state[pkg]['start_time'] <= prev_env_state_dict['time_step']
            ]
            if waiting_pkgs:
                target_pos = min(
                    ((persistent_packages_at_prev_state[pkg]['x'],
                          persistent_packages_at_prev_state[pkg]['y']) for pkg in waiting_pkgs),
                    key=lambda pos: run_bfs_distance(prev_env_state_dict['map'],(prev_r, prev_c), pos)
                )
        if target_pos and moved:
            dist_before = run_bfs_distance(prev_env_state_dict['map'],(prev_r, prev_c), target_pos)
            dist_after = run_bfs_distance(prev_env_state_dict['map'],(curr_r, curr_c), target_pos)
            if dist_after < dist_before:
                shaped_rewards[agent_idx] += SHAPING_MOVED_CLOSER_TO_TARGET
            elif dist_after > dist_before:
                shaped_rewards[agent_idx] += SHAPING_MOVED_AWAY_FROM_TARGET

        # 4. Phạt đứng yên không cần thiết
        if not moved and move_str == 'S' and prev_pkg == 0:
            idle_nearby = any(
                persistent_packages_at_prev_state[pkg]['status'] == 'waiting' and
                persistent_packages_at_prev_state[pkg]['start_time'] <= prev_env_state_dict['time_step'] and
                run_bfs_distance(prev_env_state_dict['map'],(prev_r, prev_c), (persistent_packages_at_prev_state[pkg]['x'],
                          persistent_packages_at_prev_state[pkg]['y'])) <= 3
                for pkg in persistent_packages_at_prev_state
            )
            if idle_nearby:
                shaped_rewards[agent_idx] += SHAPING_IDLE_WITH_AVAILABLE_TASKS

    return global_reward + shaped_rewards.sum()

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import time
import os # Import os for creating directories
from env import Environment # Assuming env.py is in the same directory or importable
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler
import collections # For package_dict management and episode stats deque
from tqdm import tqdm # For progress bar

# Assuming ActorCNN, CriticCNN_V3, MAPPORolloutBuffer,
# state_to_agent, convert_global_state_to_critic_input,
# convert_state_to_actor_input, compute_shaped_rewards_v2 (modified)
# are defined above this code block.

# --- Helper for modified shaped rewards (assumed to return per-agent array) ---
# IMPORTANT: Replace the placeholder below with your actual compute_shaped_rewards_v2 logic
# that returns a numpy array of shape (num_agents,).


# --- Hyperparameters ---
MAP_FILE = 'map1.txt'
MAX_TIME_STEPS = 500
NUM_ROBOTS = 5
NUM_PACKAGES = 20
ENV_SEED = 2025

# Environment rewards (less important with shaping, but still contribute)
MOVE_COST = -0.01
DELIVERY_REWARD = 10
DELAY_REWARD = 1

# Training parameters
TOTAL_TIMESTEPS = 1_000_000 # Total environment steps to run training
BUFFER_SIZE = 500         # Number of environment steps to collect per rollout
NUM_EPOCHS = 10             # Number of PPO update epochs per rollout
NUM_MINIBATCHES = 64        # Number of minibatches to split rollout data into
LEARNING_RATE_ACTOR = 5e-15
LEARNING_RATE_CRITIC = 1e-10
GAMMA = 0.99                # Discount factor
GAE_LAMBDA = 0.95           # GAE parameter
CLIP_EPSILON = 0.2          # PPO clipping parameter
ENTROPY_COEF = 0.01         # Coefficient for entropy bonus
VALUE_LOSS_COEF = 0.5       # Coefficient for value loss

# Saving parameters
SAVE_INTERVAL_EPISODES = 10 # Save model every X completed episodes
CHECKPOINT_DIR = 'mappo_checkpoints_map1' # Directory to save checkpoints

# Observation dimensions (match your converter functions)
CRITIC_OBS_CHANNELS = 7
ACTOR_OBS_CHANNELS = 6

# Map dimensions will be determined by loading the map
MAP_HEIGHT = -1
MAP_WIDTH = -1

# Device setting
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- Create Checkpoint Directory ---


# --- Initialize Environment ---
try:
    env = Environment(MAP_FILE, max_time_steps=MAX_TIME_STEPS, n_robots=NUM_ROBOTS, n_packages=NUM_PACKAGES,
                      move_cost=MOVE_COST, delivery_reward=DELIVERY_REWARD, delay_reward=DELAY_REWARD,
                      seed=ENV_SEED)
    initial_state = env.reset()
    MAP_HEIGHT = len(env.grid)
    MAP_WIDTH = len(env.grid[0]) if MAP_HEIGHT > 0 else 0
    if MAP_WIDTH == 0:
        raise ValueError("Map file is empty or invalid.")
    MAP_DIMS = (MAP_HEIGHT, MAP_WIDTH)
    print(f"Environment initialized. Map dimensions: {MAP_DIMS}")
except Exception as e:
    print(f"Error initializing environment: {e}")
    exit()

# --- Initialize Networks ---
critic = CriticCNN_V3(input_channels=CRITIC_OBS_CHANNELS).to(DEVICE)
critic.train()

actor = ActorCNN(input_channels=ACTOR_OBS_CHANNELS, num_actions=15).to(DEVICE)
actor.train()

# --- Initialize Optimizers ---
actor_optimizer = optim.Adam(actor.parameters(), lr=LEARNING_RATE_ACTOR)
critic_optimizer = optim.Adam(critic.parameters(), lr=LEARNING_RATE_CRITIC)

# --- Initialize Rollout Buffer ---
buffer = MAPPORolloutBuffer(
    buffer_size=BUFFER_SIZE,
    num_agents=NUM_ROBOTS,
    map_dims=MAP_DIMS,
    critic_obs_channels=CRITIC_OBS_CHANNELS, # Fixed typo here
    actor_obs_channels=ACTOR_OBS_CHANNELS,
    gamma=GAMMA,
    gae_lambda=GAE_LAMBDA,
    device=DEVICE
)

# --- Training Loop ---
max_env_reward = 0
total_steps_trained = 0
total_episodes_finished = 0 # Counter for completed episodes

episode_rewards = collections.deque(maxlen=100) # Track rewards for last 100 episodes
episode_durations = collections.deque(maxlen=100)
episode_delivered_packages = collections.deque(maxlen=100)
global_package_dict = {} # Persistent storage for package info across steps

current_env_state = env.get_state() # Get initial state
prev_env_state = current_env_state # Store state before action

estimated_num_training_iterations = TOTAL_TIMESTEPS // BUFFER_SIZE

print(f"Starting training (approx {TOTAL_TIMESTEPS} env steps)...")

# Use a while loop based on total_steps_trained instead of fixed iterations
# to ensure we train for approximately the target total steps.
pbar = tqdm(total=TOTAL_TIMESTEPS, desc="Total Steps Trained", unit="step")

while total_steps_trained < TOTAL_TIMESTEPS:

    # --- Data Collection (Rollout) ---
    buffer.reset() # Reset buffer for a new rollout
    rollout_start_time = time.time()
    steps_in_rollout = 0

    # Need the state AFTER the last step added to the buffer for final GAE bootstrapping
    last_global_state_obs_for_gae = None
    while buffer.ptr < BUFFER_SIZE and total_steps_trained < TOTAL_TIMESTEPS:
        current_state_for_agents, global_package_dict = state_to_agent(global_package_dict, current_env_state)

        global_obs_critic = convert_global_state_to_critic_input(
            current_state_for_agents, global_package_dict, map_dims=MAP_DIMS, max_time_steps=MAX_TIME_STEPS
        ) # Shape: (C_critic, H, W)

        agent_obs_actors = [] # List of obs for each agent
        for agent_idx in range(NUM_ROBOTS):
            local_obs_actor = convert_state_to_actor_input(
                current_state_for_agents, global_package_dict, agent_idx, map_dims=MAP_DIMS
            ) # Shape: (C_actor, H, W)
            agent_obs_actors.append(local_obs_actor)

        # --- Get Actions and Values from Networks ---
        with torch.no_grad():
            # Get value estimate for the global state
            global_obs_critic_tensor = torch.tensor(global_obs_critic, dtype=torch.float32).unsqueeze(0).to(DEVICE)
            state_value = critic(global_obs_critic_tensor).squeeze(0).item() # Get scalar value

            # Get actions and log probabilities for each agent
            actions_list = [] # List of int action indices
            log_probs_list = [] # List of log_probs
            agent_actions_for_env = [] # List of (move_str, pkg_op_str) for env.step

            for agent_idx in range(NUM_ROBOTS):
                local_obs_actor_tensor = torch.tensor(agent_obs_actors[agent_idx], dtype=torch.float32).unsqueeze(0).to(DEVICE)

                # Get action logits from the actor model for the current agent
                action_logits = actor(local_obs_actor_tensor) # Shape: (1, num_actions)

                # Sample action using Categorical distribution
                action_distribution = Categorical(logits=action_logits)
                action_tensor = action_distribution.sample() # Shape: (1,)
                action_index = action_tensor.item() # Get scalar action index

                # Get log probability of the sampled action
                log_prob_tensor = action_distribution.log_prob(action_tensor) # Shape: (1,)
                log_prob = log_prob_tensor.item() # Get scalar log_prob

                # Store results
                actions_list.append(action_index)
                log_probs_list.append(log_prob)

                # Decode action index for the environment step
                move_str, pkg_op_str = ActorCNN.decode_action(action_index) # Use the static method
                agent_actions_for_env.append((move_str, pkg_op_str))

        # --- Step the Environment ---
        # Store info needed for shaped reward calculation before the step
        prev_state_for_shaping = prev_env_state
        current_state_for_shaping = current_env_state # This will be prev_env_state *after* the step
        packages_for_shaping = global_package_dict.copy() # Use packages info *before* the step

        next_env_state, global_reward, done, infos = env.step(agent_actions_for_env)

        # --- Compute Shaped Rewards ---
        # Use the state *before* the step and the resulting state *after* the step
        # along with actions and package info from before the step.
        per_agent_shaped_rewards = compute_shaped_rewards(
             global_reward=global_reward,
             prev_env_state_dict=current_state_for_shaping, # State *before* the step
             current_env_state_dict=next_env_state,       # State *after* the step
             actions_taken_for_all_agents=agent_actions_for_env,
             persistent_packages_at_prev_state=packages_for_shaping, # Package info *before* the step
             num_agents=NUM_ROBOTS,
        )
        # The total reward for logging is the sum of per-agent rewards + any base global reward
        # Note: The per-agent rewards should ideally already incorporate the global reward logic from the env.step.
        # If compute_shaped_rewards_v2 adds its shaping ON TOP of the global_reward, you need to decide
        # if you log the sum or just the sum of the per-agent shaped rewards.
        # A simple approach is total_step_reward_for_logging = np.sum(per_agent_shaped_rewards)
        total_step_reward_for_logging = np.sum(per_agent_shaped_rewards)


        # --- Add Experience to Buffer ---
        buffer.add(
            global_obs=global_obs_critic,
            agent_obs_list=agent_obs_actors,
            action_list=actions_list,
            reward_list=per_agent_shaped_rewards, # Use per-agent rewards
            done=done,
            log_prob_list=log_probs_list,
            value=state_value
        )

        # --- Update State and Metrics ---
        prev_env_state = current_env_state # Current state becomes previous state for next step
        current_env_state = next_env_state # Next state becomes current state
        steps_in_rollout += 1
        total_steps_trained += 1
        # Update tqdm progress bar
        pbar.update(1)

        # --- Episode Handling ---
        if done:
            total_episodes_finished += 1 # Increment episode counter

            # Log episode stats (Need to accumulate them per episode first!)
            # We need episode stats accumulators *outside* this inner loop.
            # Let's add them before the while buffer.ptr loop.
            # Reset accumulators at the start of each episode.
            # Add state.total_reward to current_episode_reward before reset.
            episode_rewards.append(env.total_reward) # env.total_reward is the total for the episode
            episode_durations.append(env.t) # env.t is the duration
            delivered_in_episode = sum(1 for pkg in env.packages if pkg.status == 'delivered') # This might be state['packages'] status
            episode_delivered_packages.append(delivered_in_episode)


            print(f"\nEpisode {total_episodes_finished} finished at step {env.t} ({total_steps_trained} total). Env Reward: {env.total_reward:.2f}, Steps: {env.t}, Delivered: {delivered_in_episode}")
            max_env_reward = max(max_env_reward, env.total_reward)
            # --- Save Model Checkpoint ---
            if total_episodes_finished > 0 and total_episodes_finished % SAVE_INTERVAL_EPISODES == 0 or env.total_reward >= max_env_reward :
                 checkpoint_path = os.path.join(CHECKPOINT_DIR, f"mappo_checkpoint_ep_{total_episodes_finished:06d}_map1.pth")
                 torch.save({
                     'total_steps_trained': total_steps_trained,
                     'total_episodes_finished': total_episodes_finished,
                     'actor_state_dict': actor.state_dict(),
                     'critic_state_dict': critic.state_dict(),
                     'actor_optimizer_state_dict': actor_optimizer.state_dict(),
                     'critic_optimizer_state_dict': critic_optimizer.state_dict(),
                     'episode_rewards': list(episode_rewards), # Save deque content
                     'episode_durations': list(episode_durations),
                     'episode_delivered_packages': list(episode_delivered_packages),
                 }, checkpoint_path)
                 print(f"Saved checkpoint to {checkpoint_path}")


            # Reset environment for the next episode within the same rollout
            current_env_state = env.reset() # Reset environment
            prev_env_state = current_env_state # Store initial state as previous
            global_package_dict = {} # Clear package dict for new episode

    if not done and total_steps_trained < TOTAL_TIMESTEPS:
        current_state_for_agents, _ = state_to_agent(global_package_dict, current_env_state) # Re-process current state
        last_global_state_obs_for_gae = convert_global_state_to_critic_input(
            current_state_for_agents, global_package_dict, map_dims=MAP_DIMS, max_time_steps=MAX_TIME_STEPS
        )
    else:
        last_global_state_obs_for_gae = None # GAE bootstrap value will be 0


    # --- Compute Returns and Advantages ---
    if buffer.ptr > 0:
        buffer.compute_final_returns_and_advantages(
            last_global_state_obs=last_global_state_obs_for_gae,
            last_value_fn=critic
        )
    else:
        print("Buffer is empty, skipping GAE computation and PPO update.")
        continue


    # --- PPO Update Phase ---
    update_start_time = time.time()
    actor_loss_sum = 0
    critic_loss_sum = 0
    entropy_sum = 0
    num_minibatches_processed = 0

    minibatch_sampler = buffer.sample_minibatches(NUM_MINIBATCHES, normalize_advantages=True)

    for epoch in range(NUM_EPOCHS):
         minibatch_generator_this_epoch = buffer.sample_minibatches(NUM_MINIBATCHES, normalize_advantages=True)

         for minibatch in minibatch_generator_this_epoch:
             (
                 global_states_mb,
                 agent_local_states_mb,
                 actions_mb,
                 old_log_probs_mb,
                 old_values_mb,
                 returns_mb,
                 advantages_mb
             ) = minibatch

             # --- Critic Update ---
             critic_optimizer.zero_grad()
             predicted_values = critic(global_states_mb).squeeze(-1)
             value_loss = F.mse_loss(predicted_values, returns_mb)

             # --- Actor Update ---
             actor_optimizer.zero_grad()
             action_logits = actor(agent_local_states_mb)
             action_distribution = Categorical(logits=action_logits)
             new_log_probs = action_distribution.log_prob(actions_mb)

             ratio = torch.exp(new_log_probs - old_log_probs_mb)
             term1 = ratio * advantages_mb
             clipped_ratio = torch.clamp(ratio, 1.0 - CLIP_EPSILON, 1.0 + CLIP_EPSILON)
             term2 = clipped_ratio * advantages_mb
             actor_loss = -torch.min(term1, term2).mean()

             entropy = action_distribution.entropy().mean()
             entropy_sum += entropy.item()

             # --- Combined Loss and Optimization ---
             total_loss = actor_loss + VALUE_LOSS_COEF * value_loss - ENTROPY_COEF * entropy

             total_loss.backward()
             actor_optimizer.step()
             critic_optimizer.step()

             actor_loss_sum += actor_loss.item()
             critic_loss_sum += value_loss.item()
             num_minibatches_processed += 1

    update_end_time = time.time()
    #print(f"PPO Update finished in {update_end_time - update_start_time:.2f}s.")

    # --- Logging ---
    avg_actor_loss = actor_loss_sum / max(1, num_minibatches_processed)
    avg_critic_loss = critic_loss_sum / max(1, num_minibatches_processed)
    # Average entropy over all minibatches processed during the update phase
    avg_entropy = entropy_sum / max(1, num_minibatches_processed)


    # Print summary after update phase for this rollout
    print(f"\n--- Rollout Summary (Steps {total_steps_trained - steps_in_rollout + 1} to {total_steps_trained}) ---")
    # print(f"Rollout Duration: {rollout_end_time - rollout_start_time:.2f}s")
    print(f"Update Duration: {update_end_time - update_start_time:.2f}s")
    if episode_rewards:
        print(f"Avg Episode Reward (last {len(episode_rewards)}): {np.mean(episode_rewards):.2f}")
        print(f"Avg Episode Duration (last {len(episode_durations)}): {np.mean(episode_durations):.2f}")
        print(f"Avg Delivered Packages (last {len(episode_delivered_packages)}): {np.mean(episode_delivered_packages):.2f}")
    print(f"Avg Actor Loss (per minibatch): {avg_actor_loss:.4f}")
    print(f"Avg Critic Loss (per minibatch): {avg_critic_loss:.4f}")
    print(f"Avg Entropy (per minibatch): {avg_entropy:.4f}")
    print("-" * 30)


pbar.close() # Close tqdm progress bar

print("Training finished.")

# Optional: Save final model
final_checkpoint_path = os.path.join(CHECKPOINT_DIR, f"mappo_final_ep_{total_episodes_finished:06d}_steps_{total_steps_trained}.pth")
torch.save({
    'total_steps_trained': total_steps_trained,
    'total_episodes_finished': total_episodes_finished,
    'actor_state_dict': actor.state_dict(),
    'critic_state_dict': critic.state_dict(),
    'actor_optimizer_state_dict': actor_optimizer.state_dict(),
    'critic_optimizer_state_dict': critic_optimizer.state_dict(),
    'episode_rewards': list(episode_rewards),
    'episode_durations': list(episode_durations),
    'episode_delivered_packages': list(episode_delivered_packages),
}, final_checkpoint_path)
print(f"Saved final checkpoint to {final_checkpoint_path}")

# --- You can add evaluation code here after training ---

Using device: cuda
Environment initialized. Map dimensions: (10, 10)
Starting training (approx 1000000 env steps)...


Total Steps Trained:   0%|          | 488/1000000 [00:05<2:18:31, 120.25step/s]


Episode 1 finished at step 500 (500 total). Env Reward: -9.44, Steps: 500, Delivered: 5


Total Steps Trained:   0%|          | 512/1000000 [00:10<26:08:12, 10.62step/s]


--- Rollout Summary (Steps 1 to 500) ---
Update Duration: 4.98s
Avg Episode Reward (last 1): -9.44
Avg Episode Duration (last 1): 500.00
Avg Delivered Packages (last 1): 5.00
Avg Actor Loss (per minibatch): -0.0006
Avg Critic Loss (per minibatch): 478.5330
Avg Entropy (per minibatch): 2.5603
------------------------------


...


--- Rollout Summary (Steps 999001 to 999500) ---
Update Duration: 4.56s
Avg Episode Reward (last 100): -7.60
Avg Episode Duration (last 100): 500.00
Avg Delivered Packages (last 100): 5.65
Avg Actor Loss (per minibatch): -0.0016
Avg Critic Loss (per minibatch): 422.3163
Avg Entropy (per minibatch): 2.5598
------------------------------


Total Steps Trained: 100%|█████████▉| 999999/1000000 [4:55:33<00:00, 114.56step/s]


Episode 2000 finished at step 500 (1000000 total). Env Reward: -10.17, Steps: 500, Delivered: 4
Saved checkpoint to mappo_checkpoints_map1/mappo_checkpoint_ep_002000_map1.pth


Total Steps Trained: 100%|██████████| 1000000/1000000 [4:55:37<00:00, 56.38step/s]


--- Rollout Summary (Steps 999501 to 1000000) ---
Update Duration: 4.68s
Avg Episode Reward (last 100): -7.58
Avg Episode Duration (last 100): 500.00
Avg Delivered Packages (last 100): 5.66
Avg Actor Loss (per minibatch): 0.0019
Avg Critic Loss (per minibatch): 620.1758
Avg Entropy (per minibatch): 2.5592
------------------------------
Training finished.
Saved final checkpoint to mappo_checkpoints_map1/mappo_final_ep_002000_steps_1000000.pth
